You can run the server locally using the following command:

docker run -p 1042:1042 -it --rm -v /Users/lorrainesaju/flowR:/data eagleoutice/flowr --server

(Here, "/Users/lorrainesaju/flowR" is the directory I have chosen to mount to the container. Replace it with teh directory you want to use.)

In [49]:
import socket
import json

1. Example from the flowR wiki: (https://github.com/flowr-analysis/flowr/wiki/Interface#-ways-of-connecting)

In [15]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(('127.0.0.1', 1042))
    print(s.recv(4096))  # for the hello message

    s.send(b'{"type":"request-file-analysis","content":"x <- 1","id":"1"}\n')

    print(s.recv(65536))  # for the response (please use a more sophisticated mechanism)

b'{"type":"hello","clientName":"client-3","versions":{"flowr":"2.1.7","r":"4.3.1"}}\n'
b'{"type":"response-file-analysis","format":"json","id":"1","results":{"parse":{"parsed":"[1,1,1,6,7,0,\\"expr\\",false,\\"x <- 1\\"],[1,1,1,1,1,3,\\"SYMBOL\\",true,\\"x\\"],[1,1,1,1,3,7,\\"expr\\",false,\\"x\\"],[1,3,1,4,2,7,\\"LEFT_ASSIGN\\",true,\\"<-\\"],[1,6,1,6,4,5,\\"NUM_CONST\\",true,\\"1\\"],[1,6,1,6,5,7,\\"expr\\",false,\\"1\\"]",".meta":{"timing":3}},"normalize":{"ast":{"type":"RExpressionList","children":[{"type":"RBinaryOp","location":[1,3,1,4],"lhs":{"type":"RSymbol","location":[1,1,1,1],"content":"x","lexeme":"x","info":{"fullRange":[1,1,1,1],"additionalTokens":[],"id":0,"parent":2,"role":"binop-lhs","index":0,"nesting":0,"file":"/tmp/tmp-1-ypxhvldCWmBh-.R"}},"rhs":{"location":[1,6,1,6],"lexeme":"1","info":{"fullRange":[1,6,1,6],"additionalTokens":[],"id":1,"parent":2,"role":"binop-rhs","index":1,"nesting":0,"file":"/tmp/tmp-1-ypxhvldCWmBh-.R"},"type":"RNumber","content":{"num":1,"comp

2. Trying to get the response from the server for a file instead of directly passing the content. (REQUEST-FILE-ANALYSIS)

In [55]:
def receive_all(s, buffer_size=65536):
    response = b""
    while True:
        chunk = s.recv(buffer_size)
        response += chunk
        # Check if the chunk size is smaller than the buffer size, indicating end of response
        if len(chunk) < buffer_size:
            break
    return response

def request_file_analysis(s, filePath):
    request = {
        "type": "request-file-analysis",
        "filepath": filePath,
        "id": "1"
    }
    s.send((json.dumps(request) + "\n").encode('utf-8'))
    return receive_all(s)

b'{"type":"hello","clientName":"client-22","versions":{"flowr":"2.1.7","r":"4.3.1"}}\n'
Response Length: 23887
Raw Response (partial): b'{"type":"response-file-analysis","format":"json","id":"1","results":{"parse":{"parsed":"[1,1,1,10,10,0,\\"expr\\",false,\\"library(x)\\"],[1,1,1,7,1,3,\\"SYMBOL_FUNCTION_CALL\\",true,\\"library\\"],[1,1,1,7,3,10,\\"expr\\",false,\\"library\\"],[1,8,1,8,2,10,\\"\'(\'\\",true,\\"(\\"],[1,9,1,9,4,6,\\"SYMBOL\\",true,\\"x\\"],[1,9,1,9,6,10,\\"expr\\",false,\\"x\\"],[1,10,1,10,5,10,\\"\')\'\\",true,\\")\\"]",".meta":{"timing":5}},"normalize":{"ast":{"type":"RExpressionList","children":[{"type":"RFunctionCall","named":true,"location":[1,1,1,7],"lexeme":"library","functionName":{"type":"RSymbol","location":[1,1,1,7],"content":"library","lexeme":"library","info":{"fullRange":[1,1,1,10],"additionalTokens":[],"id":0,"parent":3,"role":"call-name","index":0,"nesting":0,"file":"file:///data/example_2.R"}},"arguments":[{"type":"RArgument","location":[1,9,1,9],"lexe

In [ ]:
# Connect to the FlowR server
HOST = "127.0.0.1"  
PORT = 1042        

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    print(s.recv(4096))  # Initial server response, typically a greeting
    
    filePath = "file:///data/example_2.R"  
    file_analysis_response = request_file_analysis(s, filePath)
    
    # Print the length of the raw response to check if it matches the expected size
    print("Response Length:", len(file_analysis_response))
    print("Raw Response (partial):", file_analysis_response[:1000])  # Print first 1000 bytes for inspection
    
    try:
        # Try decoding the JSON response
        json_data = json.loads(file_analysis_response.decode('utf-8'))
        print(json_data)  # Print the decoded JSON data
    except json.JSONDecodeError as e:
        print("JSON decoding error:", e)
        print("Problematic data (partial):", file_analysis_response[:1000])  # Print first 1000 bytes to see if there are issues


In [56]:
def visualize_keys(data, level=0):
    """Recursively print the keys of the JSON data."""
    if isinstance(data, dict):  # Check if it's a dictionary
        for key in data:
            print("  " * level + key)  # Print the key, with indentation based on its depth
            visualize_keys(data[key], level + 1)  # Recursively call for nested dictionaries
    elif isinstance(data, list):  # Check if it's a list
        for i, item in enumerate(data):
            print("  " * level + f"[{i}]")  # Print the index of the list item
            visualize_keys(item, level + 1)  # Recursively call for items in the list

# Step 4: Visualize the keys
visualize_keys(json_data)

type
format
id
results
  parse
    parsed
    .meta
      timing
  normalize
    ast
      type
      children
        [0]
          type
          named
          location
            [0]
            [1]
            [2]
            [3]
          lexeme
          functionName
            type
            location
              [0]
              [1]
              [2]
              [3]
            content
            lexeme
            info
              fullRange
                [0]
                [1]
                [2]
                [3]
              additionalTokens
              id
              parent
              role
              index
              nesting
              file
          arguments
            [0]
              type
              location
                [0]
                [1]
                [2]
                [3]
              lexeme
              value
                type
                location
                  [0]
                  [1]
               

3. Trying to replicate CLI behavior using the server. (QUERY)

In [57]:
import subprocess
import re

def run_docker_flowr(query):
    docker_command = [
        "docker", "run", "-i", "--rm",
        "-v", "/Users/lorrainesaju/flowR:/data",
        "eagleoutice/flowr"
    ]

    query_command = f':query "[{{ \\"type\\": \\"{query}\\" }}]" file:///data/example_script.R'
    print(f"Query command: {query_command}")

    try:
        # Run the Docker command
        process = subprocess.Popen(
            docker_command,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        # Send query command followed by exit command
        stdout, stderr = process.communicate(input=f"{query_command}\nexit\n")

        # Print raw outputs for debugging
        if stdout:
            print("Raw Output:", stdout)
        if stderr:
            print("Error Output:", stderr)

    except Exception as e:
        print(f"Error running Docker command: {e}")

# Example query
query = "dependencies"
run_docker_flowr(query)


Query command: :query "[{ \"type\": \"dependencies\" }]" file:///data/example_script.R
Raw Output: flowR repl using flowR 2.1.7, R 4.3.1
R> :query "[{ \"type\": \"dependencies\" }]" file:///data/example_script.R
exit
Query: dependencies (15 ms)
   ╰ Libraries
       ╰ `loadNamespace`
           ╰ Node Id: 8, `bar`
       ╰ `::`
           ╰ Node Id: 32, `better`
   ╰ Sourced Files
       ╰ `source`
           ╰ Node Id: 3, `sample.R`
   ╰ Read Data
       ╰ `read.csv`
           ╰ Node Id: 14, `data.csv`
   ╰ Written Data
       ╰ `write.csv`
           ╰ Node Id: 37, `data2.csv`
       ╰ `print`
           ╰ Node Id: 41, `stdout`
All queries together required ≈15 ms (1ms accuracy, total 67 ms)

Error Output: WARNING: The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
node:internal/readline/interface:397
      throw new ERR_USE_AFTER_CLOSE('readline');
            ^

Error [ERR_USE_AFTER_CLOSE]:

4. What we need: Communicating with the server (like in case 2) but with the query command (like in case 3), to get a JSON response that can be parsed to extract the dependecies. 